In [1]:
import os
import numpy as np
import pandas as pd
from recbole.quick_start import run_recbole
import time
import datetime
#tensor factorization
#source: https://tensorly.org/stable/user_guide/tensor_decomposition.html
import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.decomposition import tucker

from tensorly.decomposition import non_negative_tucker
from tensorly.decomposition import non_negative_parafac

C:\Users\Yiran\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-31 21:40:58,045	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-31 21:40:58,478	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
import torch

[sequential recommendation parameter guidance](https://recbole.io/docs/get_started/started/sequential.html)



In [3]:
path = "G:/My Drive/2021/Bias/sumo_simulation/"
os.chdir(path)

#get obs data matrix
data_path_lst = []
for i in os.listdir():
    if len(i) == 12 and '.csv' in i:
        data_path_lst.append(i)
#data_path_lst

## Tensor factorization test

In [9]:
test_dt = pd.read_csv(data_path_lst[0])
timestep_convert = lambda x: time.mktime(datetime.datetime.strptime(('2017_4_'+x[:-4]),
                                                                    "%Y_%m_%d_%H_%M").timetuple())
test_dt['date_time'] = test_dt['key'].apply(timestep_convert)

def convert_idx(selectcol, newcol, data):
    newdict = {}
    idx = 0
    for i in np.unique(data[selectcol]):
        newdict[i] = idx
        idx += 1
    add_val = lambda x: newdict[x]
    data[newcol] = data[selectcol].apply(add_val)
    
def calculate_rmse_mae(original, reconstructed):
    # Flatten the tensors to compute the errors
    original_flat = original.flatten()
    reconstructed_flat = reconstructed.flatten()
    
    # Calculate RMSE
    rmse = np.sqrt(np.mean((original_flat - reconstructed_flat) ** 2))
    
    # Calculate MAE
    mae = np.mean(np.abs(original_flat - reconstructed_flat))
    
    return rmse, mae

In [10]:
#convert data to 3d matrix
num_ids, num_taz, num_time = len(np.unique(test_dt.newid)), len(np.unique(test_dt.taz)), len(np.unique(test_dt.time))

In [11]:
convert_idx('newid', 'newidx', test_dt)
convert_idx('taz', 'tazidx', test_dt)
convert_idx('date_time', 'timeidx', test_dt)

In [12]:
test_mx = np.zeros((num_ids, num_taz, num_time))

for i, j, k in test_dt[['newidx', 'tazidx', 'timeidx']].values:
    test_mx[i][j][k] = 1

In [13]:
test_mx = tl.tensor(test_mx)

KeyboardInterrupt: 

In [8]:
factors = non_negative_parafac(test_mx, rank=100)

In [9]:
reconstructed_tensor = tl.kruskal_to_tensor(factors)

# Comparing the original tensor with the reconstructed tensor
reconstruction_error = tl.norm(test_mx - reconstructed_tensor) / tl.norm(test_mx)
print(f"Reconstruction error: {reconstruction_error}")
print(f"RMSE, MAE: {calculate_rmse_mae(test_mx, reconstructed_tensor)}")

Reconstruction error: 0.9441467747462673
RMSE, MAE: (0.017525987060110774, 0.0005588863170128765)


In [10]:
core, factors_tucker = tucker(test_mx, rank=[50, 24, 10])

In [11]:
reconstructed_tensor = tl.tucker_tensor.tucker_to_tensor((core, factors_tucker))
reconstruction_error = tl.norm(test_mx - reconstructed_tensor) / tl.norm(test_mx)
print(f"Reconstruction error: {reconstruction_error}")
print(f"RMSE, MAE: {calculate_rmse_mae(test_mx, reconstructed_tensor)}")

Reconstruction error: 0.9707548785143651
RMSE, MAE: (0.01801990738564394, 0.0006950310438924797)


## recbole-based
### Sequential model test

In [ ]:
#reference source: https://www.kaggle.com/code/astrung/recbole-lstm-sequential-for-recomendation-tutorial

In [52]:
temp = test_dt[['newid','taz','date_time']].rename(columns={'newid': 'user_id:token', 
                                                     'taz': 'item_id:token', 
                                                     'date_time': 'timestamp:float'})
temp.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

## Model test

In [15]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.model.sequential_recommender import FPMC, BERT4Rec, SASRec, FOSSIL

In [16]:
from recbole.quick_start import load_data_and_model
from recbole.config import Config
from recbole.data import create_dataset, data_preparation

In [9]:
#os.listdir('../SUMO_simulation/')

In [20]:
parameter_dict_FPMC = {
    'data_path': '../SUMO_simulation/',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    #'train_neg_sample_args': None,
    'checkpoint_dir': './model_checkpoints',
    'neg_sampling':None,
    'eval_args':{
        'topk':30,
        'order':'TO'
    }
}


config =  Config(model='FPMC', dataset='recbox_data', config_dict=parameter_dict_FPMC)

init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
logger.info(config)

dataset = create_dataset(config)
logger.info(dataset)
train_data, valid_data, test_data = data_preparation(config, dataset)

28 May 12:32    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ../SUMO_simulation/recbox_data
checkpoint_dir = ./model_checkpoints
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}, 'topk': 30}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decima

In [22]:
#current model
os.listdir('model_checkpoints/')

['model_FPMC.pth',
 'model_SASrec.pth',
 'model_BERT4Rec.pth',
 'model_FOSSIL.pth']

In [27]:
model = torch.load('model_checkpoints/model_FPMC.pth')

In [29]:
model

OrderedDict([('UI_emb.weight',
              tensor([[ 0.0199, -0.0053, -0.0153,  ..., -0.0236, -0.0017, -0.0016],
                      [-0.0640,  0.0404, -0.0244,  ...,  0.0406,  0.0172, -0.0889],
                      [-0.0919, -0.1722, -0.1223,  ...,  0.1103,  0.0787,  0.0875],
                      ...,
                      [-0.0905,  0.0410,  0.1475,  ..., -0.1536,  0.1417, -0.0563],
                      [ 0.0845,  0.1662,  0.0443,  ...,  0.0120, -0.1688, -0.0409],
                      [-0.0050,  0.1509, -0.1128,  ...,  0.1383,  0.0628, -0.0980]],
                     device='cuda:0')),
             ('IU_emb.weight',
              tensor([[ 0.0702,  0.0839,  0.0188,  ..., -0.0573, -0.0408, -0.1000],
                      [-0.1461,  0.8167, -0.2362,  ...,  0.4133, -0.6238, -0.7000],
                      [-0.3645,  0.4812, -0.9058,  ..., -0.1218,  0.2447,  0.2747],
                      ...,
                      [-0.5439,  0.0585, -0.0822,  ..., -0.4391, -0.3355,  0.2907],
   

In [19]:
model = load_data_and_model('')

Looking in indexes: https://mirrors.ustc.edu.cn/pypi/web/simple
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [13]:
#run_recbole(model='FPMC', dataset='recbox_data', config_dict=parameter_dict)
# Step 4: Initialization
init_seed(config['seed'], config['reproducibility'])
logger = init_logger(config)
dataset = create_dataset(config)
train_data, valid_data, test_data = data_preparation(config, dataset)

# Step 5: Training
model_FPMC = FPMC(config, train_data.dataset).to(config['device'])
trainer = Trainer(config, model_FPMC)

# Train and evaluate the model
best_valid_score_FPMC, best_valid_result_FPMC = trainer.fit(train_data, valid_data, saved=True)

model_save_path = config['checkpoint_dir'] + '/model_FPMC.pth'

torch.save(model.state_dict(), model_save_path)

08 May 06:33    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
08 May 06:33    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}, 'topk': 30}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}, 'topk': 30}]
08 May 06:33    INFO  epoch 0 training [time: 8.96s, train loss: 50.7117]
epoch 0 training [time: 8.96s, train loss: 50.7117]
08 May 06:33    INFO  epoch 0 evaluating [time: 0.51s, valid_score: 0.477800]
epoch 0 evaluating [time: 0.51s, valid_score: 0.477800]
08 May 06:33    INFO  

NameError: name 'torch' is not defined

In [16]:
torch.save(model_FPMC.state_dict(), model_save_path)

In [15]:
parameter_dict_SASRec = {
    'data_path': '../SUMO_simulation/',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'train_neg_sample_args': None,
    'neg_sampling':None,
    'save_model': True,  # This will save the model after training
    'checkpoint_dir': './model_checkpoints',
    'epochs': 50,
    'eval_args':{
        'topk':30,
        'order':'TO'
    }
}


config2 = Config(model='SASRec', dataset='recbox_data', config_dict=parameter_dict_SASRec)

#run_recbole(model='SASRec', dataset='recbox_data', config_dict=parameter_dict_SASRec)
init_seed(config2['seed'], config2['reproducibility'])

#logger initialization
init_logger(config2)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
logger.info(config2)

10 May 09:15    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ../SUMO_simulation/recbox_data
checkpoint_dir = ./model_checkpoints
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 50
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}, 'topk': 30}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_de

In [16]:
dataset = create_dataset(config2)
train_data, valid_data, test_data = data_preparation(config2, dataset)


# Step 5: Training
model_SASRec = SASRec(config2, train_data.dataset).to(config2['device'])
trainer = Trainer(config2, model_SASRec)

10 May 09:17    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
10 May 09:17    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}, 'topk': 30}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}, 'topk': 30}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group

In [17]:

# Train and evaluate the model
best_valid_score_SASRec, best_valid_result_SASRec = trainer.fit(train_data, valid_data, saved=True)

model_save_path = config2['checkpoint_dir'] + '/model_SASrec.pth'

torch.save(model_SASRec.state_dict(), model_save_path)

10 May 09:17    INFO  epoch 0 training [time: 14.25s, train loss: 314.9892]
epoch 0 training [time: 14.25s, train loss: 314.9892]
epoch 0 training [time: 14.25s, train loss: 314.9892]
10 May 09:17    INFO  epoch 0 evaluating [time: 0.35s, valid_score: 0.501500]
epoch 0 evaluating [time: 0.35s, valid_score: 0.501500]
epoch 0 evaluating [time: 0.35s, valid_score: 0.501500]
10 May 09:17    INFO  valid result: 
recall@10 : 0.759    mrr@10 : 0.5015    ndcg@10 : 0.5631    hit@10 : 0.759    precision@10 : 0.0759
valid result: 
recall@10 : 0.759    mrr@10 : 0.5015    ndcg@10 : 0.5631    hit@10 : 0.759    precision@10 : 0.0759
valid result: 
recall@10 : 0.759    mrr@10 : 0.5015    ndcg@10 : 0.5631    hit@10 : 0.759    precision@10 : 0.0759
10 May 09:17    INFO  Saving current: ./model_checkpoints\SASRec-May-10-2024_09-17-18.pth
Saving current: ./model_checkpoints\SASRec-May-10-2024_09-17-18.pth
Saving current: ./model_checkpoints\SASRec-May-10-2024_09-17-18.pth
10 May 09:17    INFO  epoch 1 tra

10 May 09:19    INFO  Saving current: ./model_checkpoints\SASRec-May-10-2024_09-17-18.pth
Saving current: ./model_checkpoints\SASRec-May-10-2024_09-17-18.pth
Saving current: ./model_checkpoints\SASRec-May-10-2024_09-17-18.pth
10 May 09:19    INFO  epoch 9 training [time: 13.34s, train loss: 189.8442]
epoch 9 training [time: 13.34s, train loss: 189.8442]
epoch 9 training [time: 13.34s, train loss: 189.8442]
10 May 09:19    INFO  epoch 9 evaluating [time: 0.26s, valid_score: 0.540400]
epoch 9 evaluating [time: 0.26s, valid_score: 0.540400]
epoch 9 evaluating [time: 0.26s, valid_score: 0.540400]
10 May 09:19    INFO  valid result: 
recall@10 : 0.785    mrr@10 : 0.5404    ndcg@10 : 0.599    hit@10 : 0.785    precision@10 : 0.0785
valid result: 
recall@10 : 0.785    mrr@10 : 0.5404    ndcg@10 : 0.599    hit@10 : 0.785    precision@10 : 0.0785
valid result: 
recall@10 : 0.785    mrr@10 : 0.5404    ndcg@10 : 0.599    hit@10 : 0.785    precision@10 : 0.0785
10 May 09:19    INFO  Saving current

10 May 09:21    INFO  epoch 17 evaluating [time: 0.26s, valid_score: 0.543200]
epoch 17 evaluating [time: 0.26s, valid_score: 0.543200]
epoch 17 evaluating [time: 0.26s, valid_score: 0.543200]
10 May 09:21    INFO  valid result: 
recall@10 : 0.7872    mrr@10 : 0.5432    ndcg@10 : 0.6017    hit@10 : 0.7872    precision@10 : 0.0787
valid result: 
recall@10 : 0.7872    mrr@10 : 0.5432    ndcg@10 : 0.6017    hit@10 : 0.7872    precision@10 : 0.0787
valid result: 
recall@10 : 0.7872    mrr@10 : 0.5432    ndcg@10 : 0.6017    hit@10 : 0.7872    precision@10 : 0.0787
10 May 09:21    INFO  epoch 18 training [time: 13.07s, train loss: 186.3867]
epoch 18 training [time: 13.07s, train loss: 186.3867]
epoch 18 training [time: 13.07s, train loss: 186.3867]
10 May 09:21    INFO  epoch 18 evaluating [time: 0.25s, valid_score: 0.543100]
epoch 18 evaluating [time: 0.25s, valid_score: 0.543100]
epoch 18 evaluating [time: 0.25s, valid_score: 0.543100]
10 May 09:21    INFO  valid result: 
recall@10 : 0.787

In [20]:
config2 = Config(model='BERT4Rec', dataset='recbox_data', config_dict=parameter_dict_SASRec)

#run_recbole(model='SASRec', dataset='recbox_data', config_dict=parameter_dict_SASRec)
init_seed(config2['seed'], config2['reproducibility'])
logger = init_logger(config2)
dataset = create_dataset(config2)
train_data, valid_data, test_data = data_preparation(config2, dataset)

# Step 5: Training
model_BERT = BERT4Rec(config2, train_data.dataset).to(config2['device'])
trainer = Trainer(config2, model_BERT)

# Train and evaluate the model
best_valid_score_BERT, best_valid_result_BERT = trainer.fit(train_data, valid_data, saved=True)

model_save_path = config2['checkpoint_dir'] + '/model_BERT4Rec.pth'

torch.save(model_BERT.state_dict(), model_save_path)

14 May 04:43    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
14 May 04:43    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}, 'topk': 30}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}, 'topk': 30}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group

epoch 7 evaluating [time: 2.63s, valid_score: 0.537100]
14 May 04:46    INFO  valid result: 
recall@10 : 0.7626    mrr@10 : 0.5371    ndcg@10 : 0.5911    hit@10 : 0.7626    precision@10 : 0.0763
valid result: 
recall@10 : 0.7626    mrr@10 : 0.5371    ndcg@10 : 0.5911    hit@10 : 0.7626    precision@10 : 0.0763
valid result: 
recall@10 : 0.7626    mrr@10 : 0.5371    ndcg@10 : 0.5911    hit@10 : 0.7626    precision@10 : 0.0763
14 May 04:46    INFO  Saving current: ./model_checkpoints\BERT4Rec-May-14-2024_04-43-43.pth
Saving current: ./model_checkpoints\BERT4Rec-May-14-2024_04-43-43.pth
Saving current: ./model_checkpoints\BERT4Rec-May-14-2024_04-43-43.pth
14 May 04:46    INFO  epoch 8 training [time: 18.69s, train loss: 160.4913]
epoch 8 training [time: 18.69s, train loss: 160.4913]
epoch 8 training [time: 18.69s, train loss: 160.4913]
14 May 04:46    INFO  epoch 8 evaluating [time: 2.75s, valid_score: 0.539600]
epoch 8 evaluating [time: 2.75s, valid_score: 0.539600]
epoch 8 evaluating [t

Saving current: ./model_checkpoints\BERT4Rec-May-14-2024_04-43-43.pth
14 May 04:49    INFO  epoch 16 training [time: 18.66s, train loss: 156.2799]
epoch 16 training [time: 18.66s, train loss: 156.2799]
epoch 16 training [time: 18.66s, train loss: 156.2799]
14 May 04:49    INFO  epoch 16 evaluating [time: 2.33s, valid_score: 0.547300]
epoch 16 evaluating [time: 2.33s, valid_score: 0.547300]
epoch 16 evaluating [time: 2.33s, valid_score: 0.547300]
14 May 04:49    INFO  valid result: 
recall@10 : 0.7811    mrr@10 : 0.5473    ndcg@10 : 0.6032    hit@10 : 0.7811    precision@10 : 0.0781
valid result: 
recall@10 : 0.7811    mrr@10 : 0.5473    ndcg@10 : 0.6032    hit@10 : 0.7811    precision@10 : 0.0781
valid result: 
recall@10 : 0.7811    mrr@10 : 0.5473    ndcg@10 : 0.6032    hit@10 : 0.7811    precision@10 : 0.0781
14 May 04:49    INFO  Saving current: ./model_checkpoints\BERT4Rec-May-14-2024_04-43-43.pth
Saving current: ./model_checkpoints\BERT4Rec-May-14-2024_04-43-43.pth
Saving current:

14 May 04:56    INFO  epoch 35 training [time: 18.82s, train loss: 149.7253]
epoch 35 training [time: 18.82s, train loss: 149.7253]
epoch 35 training [time: 18.82s, train loss: 149.7253]
14 May 04:56    INFO  epoch 35 evaluating [time: 2.53s, valid_score: 0.550200]
epoch 35 evaluating [time: 2.53s, valid_score: 0.550200]
epoch 35 evaluating [time: 2.53s, valid_score: 0.550200]
14 May 04:56    INFO  valid result: 
recall@10 : 0.7868    mrr@10 : 0.5502    ndcg@10 : 0.6068    hit@10 : 0.7868    precision@10 : 0.0787
valid result: 
recall@10 : 0.7868    mrr@10 : 0.5502    ndcg@10 : 0.6068    hit@10 : 0.7868    precision@10 : 0.0787
valid result: 
recall@10 : 0.7868    mrr@10 : 0.5502    ndcg@10 : 0.6068    hit@10 : 0.7868    precision@10 : 0.0787
14 May 04:56    INFO  epoch 36 training [time: 18.99s, train loss: 146.1932]
epoch 36 training [time: 18.99s, train loss: 146.1932]
epoch 36 training [time: 18.99s, train loss: 146.1932]
14 May 04:56    INFO  epoch 36 evaluating [time: 2.71s, vali

14 May 05:00    INFO  epoch 45 training [time: 18.92s, train loss: 147.1604]
epoch 45 training [time: 18.92s, train loss: 147.1604]
epoch 45 training [time: 18.92s, train loss: 147.1604]
14 May 05:00    INFO  epoch 45 evaluating [time: 2.48s, valid_score: 0.551800]
epoch 45 evaluating [time: 2.48s, valid_score: 0.551800]
epoch 45 evaluating [time: 2.48s, valid_score: 0.551800]
14 May 05:00    INFO  valid result: 
recall@10 : 0.7868    mrr@10 : 0.5518    ndcg@10 : 0.608    hit@10 : 0.7868    precision@10 : 0.0787
valid result: 
recall@10 : 0.7868    mrr@10 : 0.5518    ndcg@10 : 0.608    hit@10 : 0.7868    precision@10 : 0.0787
valid result: 
recall@10 : 0.7868    mrr@10 : 0.5518    ndcg@10 : 0.608    hit@10 : 0.7868    precision@10 : 0.0787
14 May 05:00    INFO  epoch 46 training [time: 18.73s, train loss: 149.3211]
epoch 46 training [time: 18.73s, train loss: 149.3211]
epoch 46 training [time: 18.73s, train loss: 149.3211]
14 May 05:00    INFO  epoch 46 evaluating [time: 2.53s, valid_s

In [21]:
config2 = Config(model='FOSSIL', dataset='recbox_data', config_dict=parameter_dict_SASRec)

#run_recbole(model='SASRec', dataset='recbox_data', config_dict=parameter_dict_SASRec)
init_seed(config2['seed'], config2['reproducibility'])
logger = init_logger(config2)
dataset = create_dataset(config2)
train_data, valid_data, test_data = data_preparation(config2, dataset)

# Step 5: Training
model_fossil = FOSSIL(config2, train_data.dataset).to(config2['device'])
trainer = Trainer(config2, model_fossil)

# Train and evaluate the model
best_valid_score_fossil, best_valid_result_fossil = trainer.fit(train_data, valid_data, saved=True)

model_save_path = config2['checkpoint_dir'] + '/model_FOSSIL.pth'

torch.save(model_fossil.state_dict(), model_save_path)

14 May 05:01    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}]
14 May 05:01    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}, 'topk': 30}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}, 'topk': 30}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group

14 May 05:02    INFO  valid result: 
recall@10 : 0.7405    mrr@10 : 0.5187    ndcg@10 : 0.5717    hit@10 : 0.7405    precision@10 : 0.074
valid result: 
recall@10 : 0.7405    mrr@10 : 0.5187    ndcg@10 : 0.5717    hit@10 : 0.7405    precision@10 : 0.074
valid result: 
recall@10 : 0.7405    mrr@10 : 0.5187    ndcg@10 : 0.5717    hit@10 : 0.7405    precision@10 : 0.074
14 May 05:02    INFO  Saving current: ./model_checkpoints\FOSSIL-May-14-2024_05-01-36.pth
Saving current: ./model_checkpoints\FOSSIL-May-14-2024_05-01-36.pth
Saving current: ./model_checkpoints\FOSSIL-May-14-2024_05-01-36.pth
14 May 05:02    INFO  epoch 8 training [time: 3.18s, train loss: 183.6258]
epoch 8 training [time: 3.18s, train loss: 183.6258]
epoch 8 training [time: 3.18s, train loss: 183.6258]
14 May 05:02    INFO  epoch 8 evaluating [time: 0.11s, valid_score: 0.520400]
epoch 8 evaluating [time: 0.11s, valid_score: 0.520400]
epoch 8 evaluating [time: 0.11s, valid_score: 0.520400]
14 May 05:02    INFO  valid resul

14 May 05:03    INFO  epoch 24 evaluating [time: 0.12s, valid_score: 0.538000]
epoch 24 evaluating [time: 0.12s, valid_score: 0.538000]
epoch 24 evaluating [time: 0.12s, valid_score: 0.538000]
14 May 05:03    INFO  valid result: 
recall@10 : 0.7676    mrr@10 : 0.538    ndcg@10 : 0.5929    hit@10 : 0.7676    precision@10 : 0.0768
valid result: 
recall@10 : 0.7676    mrr@10 : 0.538    ndcg@10 : 0.5929    hit@10 : 0.7676    precision@10 : 0.0768
valid result: 
recall@10 : 0.7676    mrr@10 : 0.538    ndcg@10 : 0.5929    hit@10 : 0.7676    precision@10 : 0.0768
14 May 05:03    INFO  Saving current: ./model_checkpoints\FOSSIL-May-14-2024_05-01-36.pth
Saving current: ./model_checkpoints\FOSSIL-May-14-2024_05-01-36.pth
Saving current: ./model_checkpoints\FOSSIL-May-14-2024_05-01-36.pth
14 May 05:03    INFO  epoch 25 training [time: 3.18s, train loss: 172.3060]
epoch 25 training [time: 3.18s, train loss: 172.3060]
epoch 25 training [time: 3.18s, train loss: 172.3060]
14 May 05:03    INFO  epoch

valid result: 
recall@10 : 0.7729    mrr@10 : 0.5411    ndcg@10 : 0.5965    hit@10 : 0.7729    precision@10 : 0.0773
14 May 05:03    INFO  Saving current: ./model_checkpoints\FOSSIL-May-14-2024_05-01-36.pth
Saving current: ./model_checkpoints\FOSSIL-May-14-2024_05-01-36.pth
Saving current: ./model_checkpoints\FOSSIL-May-14-2024_05-01-36.pth
14 May 05:03    INFO  epoch 33 training [time: 3.20s, train loss: 170.5127]
epoch 33 training [time: 3.20s, train loss: 170.5127]
epoch 33 training [time: 3.20s, train loss: 170.5127]
14 May 05:03    INFO  epoch 33 evaluating [time: 0.11s, valid_score: 0.542200]
epoch 33 evaluating [time: 0.11s, valid_score: 0.542200]
epoch 33 evaluating [time: 0.11s, valid_score: 0.542200]
14 May 05:03    INFO  valid result: 
recall@10 : 0.7732    mrr@10 : 0.5422    ndcg@10 : 0.5974    hit@10 : 0.7732    precision@10 : 0.0773
valid result: 
recall@10 : 0.7732    mrr@10 : 0.5422    ndcg@10 : 0.5974    hit@10 : 0.7732    precision@10 : 0.0773
valid result: 
recall@10

In [53]:
import torch

In [55]:
user = torch.tensor([test_dt.newid[0]])  # User ID in tensor
item = torch.tensor([test_dt.taz[0]])

model_FPMC.predict(user, item)
# Item ID in tensor
#model_FPMC

device(type='cpu')

In [79]:
init_seed(config['seed'], config['reproducibility'])

23 Apr 11:45    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = ../SUMO_simulation/recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 300
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'none', 'sample_num': 'none', 'alpha': 'none', 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [0.8, 0.1, 0.1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = True
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset 

23 Apr 21:32    INFO  recbox_data
The number of users: 13710
Average actions of users: 16.57276241884893
The number of items: 168
Average actions of items: 1360.4550898203593
The number of inters: 227196
The sparsity of the dataset: 90.13597999374805%
Remain Fields: ['user_id', 'item_id', 'timestamp']
recbox_data
The number of users: 13710
Average actions of users: 16.57276241884893
The number of items: 168
Average actions of items: 1360.4550898203593
The number of inters: 227196
The sparsity of the dataset: 90.13597999374805%
Remain Fields: ['user_id', 'item_id', 'timestamp']
23 Apr 21:32    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
23 Apr 21:32    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{

In [35]:
from recbole.model.sequential_recommender import SASRec, GRU4Rec

In [36]:
# model loading and initialization
model = SASRec(config2, train_data.dataset).to(config2['device'])
logger.info(model)

# trainer loading and initialization
trainer = Trainer(config, model)

# model training
best_valid_score, best_valid_result = trainer.fit(train_data)

23 Apr 21:33    INFO  SASRec(
  (item_embedding): Embedding(168, 64, padding_idx=0)
  (position_embedding): Embedding(50, 64)
  (trm_encoder): TransformerEncoder(
    (layer): ModuleList(
      (0-1): 2 x TransformerLayer(
        (multi_head_attention): MultiHeadAttention(
          (query): Linear(in_features=64, out_features=64, bias=True)
          (key): Linear(in_features=64, out_features=64, bias=True)
          (value): Linear(in_features=64, out_features=64, bias=True)
          (softmax): Softmax(dim=-1)
          (attn_dropout): Dropout(p=0.5, inplace=False)
          (dense): Linear(in_features=64, out_features=64, bias=True)
          (LayerNorm): LayerNorm((64,), eps=1e-12, elementwise_affine=True)
          (out_dropout): Dropout(p=0.5, inplace=False)
        )
        (feed_forward): FeedForward(
          (dense_1): Linear(in_features=64, out_features=256, bias=True)
          (dense_2): Linear(in_features=256, out_features=64, bias=True)
          (LayerNorm): LayerNo

24 Apr 01:50    INFO  epoch 53 training [time: 280.64s, train loss: 182.0153]
epoch 53 training [time: 280.64s, train loss: 182.0153]
24 Apr 01:50    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 01:54    INFO  epoch 54 training [time: 282.00s, train loss: 181.8769]
epoch 54 training [time: 282.00s, train loss: 181.8769]
24 Apr 01:54    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 01:59    INFO  epoch 55 training [time: 281.05s, train loss: 181.8251]
epoch 55 training [time: 281.05s, train loss: 181.8251]
24 Apr 01:59    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 02:04    INFO  epoch 56 training [time: 280.89s, train loss: 181.6839]
epoch 56 training [time: 280.89s, train loss: 181.6839]
24 Apr 02:04    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving curre

24 Apr 04:16    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 04:21    INFO  epoch 85 training [time: 285.41s, train loss: 180.3197]
epoch 85 training [time: 285.41s, train loss: 180.3197]
24 Apr 04:21    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 04:26    INFO  epoch 86 training [time: 292.41s, train loss: 180.4908]
epoch 86 training [time: 292.41s, train loss: 180.4908]
24 Apr 04:26    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 04:31    INFO  epoch 87 training [time: 294.10s, train loss: 180.3565]
epoch 87 training [time: 294.10s, train loss: 180.3565]
24 Apr 04:31    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 04:35    INFO  epoch 88 training [time: 279.15s, train loss: 180.3680]
epoch 88 trainin

24 Apr 06:47    INFO  epoch 116 training [time: 287.62s, train loss: 179.6055]
epoch 116 training [time: 287.62s, train loss: 179.6055]
24 Apr 06:47    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 06:52    INFO  epoch 117 training [time: 282.38s, train loss: 179.5195]
epoch 117 training [time: 282.38s, train loss: 179.5195]
24 Apr 06:52    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 06:57    INFO  epoch 118 training [time: 280.44s, train loss: 179.5949]
epoch 118 training [time: 280.44s, train loss: 179.5949]
24 Apr 06:57    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 07:02    INFO  epoch 119 training [time: 280.50s, train loss: 179.5956]
epoch 119 training [time: 280.50s, train loss: 179.5956]
24 Apr 07:02    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Savi

24 Apr 11:40    INFO  epoch 178 training [time: 283.63s, train loss: 178.4063]
epoch 178 training [time: 283.63s, train loss: 178.4063]
24 Apr 11:40    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 11:44    INFO  epoch 179 training [time: 282.91s, train loss: 178.5323]
epoch 179 training [time: 282.91s, train loss: 178.5323]
24 Apr 11:44    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 11:49    INFO  epoch 180 training [time: 281.58s, train loss: 178.5496]
epoch 180 training [time: 281.58s, train loss: 178.5496]
24 Apr 11:49    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
24 Apr 11:54    INFO  epoch 181 training [time: 281.04s, train loss: 178.4223]
epoch 181 training [time: 281.04s, train loss: 178.4223]
24 Apr 11:54    INFO  Saving current: saved\FPMC-Apr-23-2024_21-33-34.pth
Savi

RuntimeError: Parent directory saved does not exist.

    self.run()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\site-packages\tensorboard\summary\writer\event_file_writer.py", line 244, in run
    

In [ ]:
# model loading and initialization
model2 = GRU4Rec(config2, train_data.dataset).to(config2['device'])
logger.info(model2)

# trainer loading and initialization
trainer = Trainer(config, model2)

# model training
best_valid_score_gru4rec, best_valid_result_gru4rec = trainer.fit(train_data)

self._run()
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\site-packages\tensorboard\summary\writer\event_file_writer.py", line 275, in _run
    self._record_writer.write(data)
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\site-packages\tensorboard\summary\writer\record_writer.py", line 40, in write
    self._writer.write(header + header_crc + data + footer_crc)
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\site-packages\tensorboard\compat\tensorflow_stub\io\gfile.py", line 773, in write
    self.fs.append(self.filename, file_content, self.binary_mode)
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\site-packages\tensorboard\compat\tensorflow_stub\io\gfile.py", line 167, in append
    self._write(filename, file_content, "ab" if binary_mode else "a")
  File "C:\Users\Yiran\anaconda3\envs\modeltest\lib\site-packages\tensorboard\compat\tensorflow_stub\io\gfile.py", line 171, in _write
    with io.open(filename, mode, encoding=encoding) as f:
OSError: [Errno 22] Invali

In [90]:
best_valid_score

-inf

In [89]:
best_valid_result

In [86]:
from recbole.utils.case_study import full_sort_topk
external_user_ids = dataset.id2token(
    dataset.uid_field, list(range(dataset.user_num)))[1:]#fist element in array is 'PAD'(default of Recbole) ->remove it 

In [87]:
topk_items = []
for internal_user_id in list(range(dataset.user_num))[1:]:
    _, topk_iid_list = full_sort_topk([internal_user_id], model, test_data, k=12, device=config['device'])
    last_topk_iid_list = topk_iid_list[-1]
    external_item_list = dataset.id2token(dataset.iid_field, last_topk_iid_list.cpu()).tolist()
    topk_items.append(external_item_list)
print(len(topk_items))

IndexError: index -1 is out of bounds for dimension 0 with size 0

In [88]:
external_item_str = [' '.join(x) for x in topk_items]
result = pd.DataFrame(external_user_ids, columns=['customer_id'])
result['prediction'] = external_item_str
result.head()

ValueError: Length of values (3) does not match length of index (13709)